# Исследование надёжности заёмщиков — анализ банковских данных

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель проекта:** исследовать влияние семейного положения и количества детей клиента на факт погашения кредита в срок.

**Исходные данные** предоставлены в виде файла *data.csv*, содержащего статистику о платёжеспособности клиентов.

**Краткий план работы:** исследовать исходные данные; провести предобработку данных; решить задачи исследования; сделать выводы.

1. [Шаг 1. Откройте файл с данными и изучите общую информацию](#start)
    * [Чтение файла и общая информация](#read)
    * [Оценка качества данных](#quality)
2. [Шаг 2. Предобработка данных](#preprocessing)
    * [Обработка пропусков](#space)
    * [Замена типов данных](#type)
    * [Обработка дубликатов](#duplicates)
        * [Дубликаты в столбце education](#education)
        * [Дубликаты в исходной таблице](#maindf)
    * [Лемматизация](#lemmas)
    * [Категоризация данных](#categories)
        * [Категоризация целей запрашиваемых кредитов](#purposes)
        * [Категоризация дохода](#income)
3. [Шаг 3. Ответьте на вопросы](#answers)
4. [Шаг 4. Общий вывод](#conclusion)
5. [Чек-лист готовности проекта](#checklist)

<a id="start"></a>
## Шаг 1. Откройте файл с данными и изучите общую информацию

<a id='read'></a>
### Чтение файла и общая информация

Импортируем библиотеку для работы с таблицами, библиотеку для работы с математическими функциями, функцию для последующей работы с лемматизацией и функцию для подсчета словарных форм.

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from pymystem3 import Mystem
m = Mystem()

In [4]:
from collections import Counter

Открываем файл.

In [5]:
data = pd.read_csv('/datasets/data.csv')

Изучим общую информацию о данных в датафрейме. Выведем первые 5 строк таблицы, чтобы увидеть ее содержание.

In [6]:
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Применим также метод `sample()`, который генерируют выборку случайной строки из фрейма данных. Применим параметр *n=5*.

In [7]:
data.sample(n=5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
8749,1,-756.281915,38,среднее,1,женат / замужем,0,M,сотрудник,0,245258.782392,покупка недвижимости
1664,1,-3137.267886,46,среднее,1,женат / замужем,0,F,сотрудник,0,106806.896417,приобретение автомобиля
12069,2,-3486.130665,35,среднее,1,женат / замужем,0,M,сотрудник,0,146573.281878,образование
17956,0,-1442.909730,43,среднее,1,гражданский брак,1,F,сотрудник,0,95296.235004,сыграть свадьбу
20953,1,-6320.653030,35,среднее,1,женат / замужем,0,M,сотрудник,0,187064.528816,операции с жильем


Применим метод `describe()` для получения описательной статистики исходного датафрейма.

<a id='describe'></a>

In [8]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Выведем общую информацию о таблице с более точной информацией об объеме затрачиваемой памяти.

<a id='info_type'></a>

In [9]:
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 11.3 MB


В таблице 12 столбцов и 21 525 строк.

Столбцы названы верно: без пробелов, кириллицы, в змеином регистре, буквы только строчные. Их не придется переименовывать.

* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

Типы данных видны, в двух столбцах есть пропуски.

Качество данных вызывает вопросы, поэтому рассмотрим столбцы подробнее.

<a id='quality'></a>
### Оценка качества данных

<a id='child'></a>

Выведем количество данных (строк) по каждой категории.

In [10]:
print(data['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


В данных встречаются отрицательные значения. Этот столбец для цели исследования важен.
**Потребуются исправления.**

Выведем 10 случайных строк столбца стажа, чтобы лучше рассмотреть данные.

In [11]:
print(data['days_employed'].sample(n=10))

10308   -1115.883521
2013    -2108.414759
9805    -1991.395531
17471     -90.293016
5748             NaN
5084    -1773.236413
8414    -2495.464642
9958             NaN
7448    -1076.893971
18966   -1023.520929
Name: days_employed, dtype: float64


В данных столбца `days_employed` встречаются как отрицательные значения, так и положительные значения. Кроме того, значения явно аномальные. Хорошо бы уточнить их у того, кто данные предоставлял. Впрочем, этот столбец не слишком важен для цели исследования. Поэтому исправим несколько формально.

<a id='step1'></a>

In [12]:
print(data['education'].value_counts())

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64


Много повторов из-за разных регистров. Этот столбец не слишком важен для цели исследования. **Потребуются исправления.**

In [13]:
print(data['education_id'].value_counts())

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64


Соотнеся данные из двух столбцов по образованию, идентификаторы легко определяются.

In [14]:
print(data['family_status'].value_counts())

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64


Здесь всё в порядке.

In [15]:
print(data['family_status_id'].value_counts())

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64


Здесь тоже всё хорошо.

In [16]:
print(data['gender'].value_counts())

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


Здесь есть одно непонятное значение. Его можно будет смело удалить, но в другой раз, так как для целей исследования сейчас некритично.

In [17]:
print(data['income_type'].value_counts())

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64


Здесь всё нормально, хотя эти данные нам не слишком важны для цели исследования.

In [18]:
print(data['debt'].value_counts())

0    19784
1     1741
Name: debt, dtype: int64


Здесь всё верно. В наличии два значения идентификатора по задолженности: 1 - есть / была задолженность, 0 - нет.

<a id='type_purpose'></a>

In [19]:
print(data['purpose'].value_counts())

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Данные выглядят слишком разнообразными, можно будет их сгруппировать. **Потребуются изменения.**

<a id='duplicates-2'></a>
Проверим данные на явные дубликаты.

In [20]:
print('Дубликатов в таблице:', data.duplicated().sum())

Дубликатов в таблице: 54


В таблице присутствуют явные дубликаты. **Потребуются изменения.**

**Вывод**

С данными в таблице есть проблемы: присутствуют пропуски и дубликаты. Возможная причина пропусков - при подаче заявки на кредит потенциальные заемщики просто не указали свой доход и стаж.

Кроме того, часть данных без необходимости представлена в вещественном виде или с отрицательными значениями.

Таким образом, данных достаточно для анализа. Но требуется устранить проблемы в них.

<a id='preprocessing'></a>
## Шаг 2. Предобработка данных

<a id='space'></a>
### Обработка пропусков

Проверим количество пропусков в таблице.

In [21]:
print(data.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Заполним пропущенные данные для дохода и стажа. Будем брать медианные значения, так как они более точно характеризуют данную выборку по "средней" величине.

Определим медианное значение дохода.

In [22]:
median_total_income = data['total_income'].median()
print(median_total_income)

145017.93753253992


Заменим пропущенные данные в столбце по доходу на медиану.

In [23]:
data['total_income'] = data['total_income'].fillna(value = median_total_income)

Определим медианное значение по стажу. Но предварительно приведем все значения в столбце к положительным числам.

In [24]:
data['days_employed'] = data['days_employed'].apply(abs)

In [25]:
median_days_employed = data['days_employed'].median()
print(median_days_employed)

2194.220566878695


Заменим пропущенные данные в столбце по стажу на медиану.

In [26]:
data['days_employed'] = data['days_employed'].fillna(value = median_days_employed)

Проверим, что пропусков в таблице нет.

In [27]:
print(data.isna().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


**Вывод**

В таблице были пропуски количественных данных в столбцах по стажу и доходу. Методом `fillna` пропуски заполнены медианными значениями в обоих столбцах. Стоит отметить, что данные по стажу требуют уточнения и не важны для данного исследования, однако сделать их обработку не сложно, поэтому оставила.

<a id='type'></a>
### Замена типов данных

По [информации](#info_type) об исходной таблице, полученной методом `info()`, видны типы содержащихся в каждом столбце данных. Исправим их с учетом полученных [значений](#describe) для уменьшения объема памяти, затрачиваемой на хранение, а следовательно, и обработку датафрейма.

В столбце `children` содержатся значения от -1 до 20. Приведем значения к положительному виду и заменим тип данных на `uint8`.

In [28]:
data['children'] = data['children'].apply(abs).astype(np.uint8)

В столбце `days_employed`заменим тип данных на `uint32`.

In [29]:
data['days_employed'] = data['days_employed'].astype(np.uint32)

В столбце `dob_years` заменим тип данных на `uint8`.

In [30]:
data['dob_years'] = data['dob_years'].astype(np.uint8)

В столбцах `education_id` и `family_status_id` заменим тип данных на `uint8`.

In [31]:
data['education_id'] = data['education_id'].astype(np.uint8)
data['family_status_id'] = data['family_status_id'].astype(np.uint8)

В столбце `debt` заменим тип данных на `uint8`.

In [32]:
data['debt'] = data['debt'].astype(np.uint8)

В столбце `total_income` заменим тип данных на `uint32`.

In [33]:
data['total_income'] = data['total_income'].astype(np.uint32)

Для столбцов `total_income` и `days_employed` заменили данные на целочисленные, поскольку для анализа нам не важны копейки / центы / ... и часы / минуты.

In [34]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


In [35]:
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null uint8
days_employed       21525 non-null uint32
dob_years           21525 non-null uint8
education           21525 non-null object
education_id        21525 non-null uint8
family_status       21525 non-null object
family_status_id    21525 non-null uint8
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null uint8
total_income        21525 non-null uint32
purpose             21525 non-null object
dtypes: object(5), uint32(2), uint8(5)
memory usage: 12.6 MB


**Вывод**

Числовые типы данных исходного датафрейма приведены к другими типам методом `astype()` в соответствии со значениями, хранящимися в каждом столбце.

<a id='duplicates'></a>
### Обработка дубликатов

<a id='education'></a>
#### Дубликаты в столбце *education*

При первичном анализе данных на [Шаге 1](#step1) было показано, что в столбце с уровнем образования есть повторяющиеся значения из-за различного регистра. Их появление, очевидно, вызвано человеческим фактором.

Обработаем их, создав новый столбец в исходном датафрейме, в котором будут содержаться приведенные к нижнему регистру данные.

In [36]:
data['education'] = data['education'].str.lower()

 Проверим то, что сделали на предыдущем шаге.

In [37]:
print(data['education'].value_counts())

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


In [38]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


<a id='maindf'></a>
#### Дубликаты в исходной таблице

При изучении данных на [Шаге 1](#duplicates-2) были выявлены явные дубликаты. Исправим это, применив метод `drop_duplicates()`.

Удалим дубликаты и перезапишем индексы для каждой строки таблицы.

In [39]:
data = data.drop_duplicates().reset_index(drop=True)

Проверим количество дубликатов после удаления.

In [40]:
print(data.duplicated().sum())

0


Выведем информацию о таблице.

In [41]:
data.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null uint8
days_employed       21454 non-null uint32
dob_years           21454 non-null uint8
education           21454 non-null object
education_id        21454 non-null uint8
family_status       21454 non-null object
family_status_id    21454 non-null uint8
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null uint8
total_income        21454 non-null uint32
purpose             21454 non-null object
dtypes: object(5), uint32(2), uint8(5)
memory usage: 12.6 MB


**Вывод**

На шаге 2 устранены дубликаты в таблице. Явные дубликаты удалены из таблицы, а индексы строк перезаписаны. Дубликаты, связанные с регистром написания, исправлены выведением их в отдельный столбец.

<a id='lemmas'></a>
### Лемматизация

На [Шаге 1](#type_purpose) было показано, что в столбце целей запрашиваемых кредитов много разных значений.

Найдем словарные формы слов в целях запрашиваемых кредитов с помощью лемматизации. Это поможет нам в дальнейшем выделить категории целей кредитов и уменьшить их количество.

Отфильтруем столбец `purpose`, удалив все дубликаты в нем, и приведем его к списку.

In [42]:
purpose_list = data['purpose'].drop_duplicates().to_list()

Сделаем из списка строку, поскольку функция лемматизации работает со строкой.

In [43]:
purpose_string = ' '.join(purpose_list)
print(purpose_string)

покупка жилья приобретение автомобиля дополнительное образование сыграть свадьбу операции с жильем образование на проведение свадьбы покупка жилья для семьи покупка недвижимости покупка коммерческой недвижимости покупка жилой недвижимости строительство собственной недвижимости недвижимость строительство недвижимости на покупку подержанного автомобиля на покупку своего автомобиля операции с коммерческой недвижимостью строительство жилой недвижимости жилье операции со своей недвижимостью автомобили заняться образованием сделка с подержанным автомобилем получение образования автомобиль свадьба получение дополнительного образования покупка своего жилья операции с недвижимостью получение высшего образования свой автомобиль сделка с автомобилем профильное образование высшее образование покупка жилья для сдачи на покупку автомобиля ремонт жилью заняться высшим образованием


Получим словарные формы слов из целей кредитов.

In [44]:
lemmas = m.lemmatize(purpose_string)
print(lemmas)

['покупка', ' ', 'жилье', ' ', 'приобретение', ' ', 'автомобиль', ' ', 'дополнительный', ' ', 'образование', ' ', 'сыграть', ' ', 'свадьба', ' ', 'операция', ' ', 'с', ' ', 'жилье', ' ', 'образование', ' ', 'на', ' ', 'проведение', ' ', 'свадьба', ' ', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', ' ', 'покупка', ' ', 'недвижимость', ' ', 'покупка', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'покупка', ' ', 'жилой', ' ', 'недвижимость', ' ', 'строительство', ' ', 'собственный', ' ', 'недвижимость', ' ', 'недвижимость', ' ', 'строительство', ' ', 'недвижимость', ' ', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', ' ', 'на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', ' ', 'операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'строительство', ' ', 'жилой', ' ', 'недвижимость', ' ', 'жилье', ' ', 'операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', ' ', 'автомобиль', ' ', 'заниматься', ' ', 'образование', ' ', 'сделка', ' ', 'с', ' ', 'подержать

Для последующего создания категорий оценим частоту словарных форм в получившимся списке.

In [45]:
print(Counter(lemmas))

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


**Вывод**

Путем лемматизации получили словарные формы слов, упоминаемых в целях запрашиваемых кредитов. Чтобы оценить возможные категории данных, подсчитали количество упоминаний лемм.

Анализ показывает, что в дальнейшем данные по столбцу `purpose` можно разделить на категории по входящим в них словам (1) недвижимость или жилье, (2) автомобиль, (3) образование, (4) свадьба.

<a id='categories'></a>
### Категоризация данных

<a id='purposes'></a>
#### Категоризация целей запрашиваемых кредитов

Создадим функцию, которая будет распределять цели кредитов по категориям.

In [46]:
def purpose_group(purpose):
    if ('недвижим' in purpose) or ('жиль' in purpose):
        return 'недвижимость'
    if 'автомобил' in purpose:
        return 'автомобиль'
    if 'образован' in purpose:
        return 'образование'
    return 'прочее'

Применим созданную функцию к столбцу `purpose` с помощью метода `apply(func)` и создадим новый столбец с категориями.

In [47]:
data['purpose_group'] = data['purpose'].apply(purpose_group)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,прочее
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,прочее
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость


<a id='income'></a>
#### Категоризация дохода

По версии Организации экономического сотрудничества и развития средний класс обладает доходом в 75-200% от медианного значения национального дохода. Будем считать нашу выборку репрезентативной.

Тогда примем, что:
<li> низкий доход - до 75% от медианного значения;</li>
<li> средний доход - 75-200% от медианного дохода;</li>
<li> высокий доход - свыше 200% от медианного дохода.</li>

Создадим функцию, которая будет распределять доход по трем категориям.

In [48]:
def income_group(income):
    if income <= 0.75 * median_total_income:
        return 'низкий доход'
    if 0.75 * median_total_income < income <= 2 * median_total_income:
        return 'средний доход'
    if income > 2 * median_total_income:
        return 'высокий доход'

Применим созданную функцию к столбцу `total_income` с помощью метода `apply(func)` и создадим новый столбец с категориями.

In [49]:
data['income_group'] = data['total_income'].apply(income_group)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,средний доход
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,средний доход
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,средний доход
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,средний доход
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,прочее,средний доход
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,средний доход
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,средний доход
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,средний доход
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,прочее,низкий доход
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,средний доход


**Вывод**

Все цели запрашиваемых кредитов выделены в четыре категории: "недвижимость", "автомобиль", "образование" и "прочее". В "прочее" входят свадьбы и иные мероприятия, связанные с большими тратами.

Доходы потенциальных заемщиков разделены на три категории: "высокий доход", "средний доход", "низкий доход".

В обоих случаях применялись одинаковые методы и были написаны пользовательские функции.

<a id='answers'></a>
## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Сгруппируем данные по количеству детей и найдем сумму и количество невозвратов для каждой группы. Посчитаем долю невозвратов в зависимости от количества детей, т.е. долю от каждой группы.

In [50]:
children_grouped = data.groupby('children').agg({'debt': ['sum', 'count']})
children_grouped['frac'] = children_grouped['debt']['sum'] / children_grouped['debt']['count']
print(children_grouped)

            debt             frac
             sum  count          
children                         
0         1063.0  14091  0.075438
1          445.0   4855  0.091658
2          194.0   2052  0.094542
3           27.0    330  0.081818
4            4.0     41  0.097561
5            0.0      9  0.000000
20           8.0     76  0.105263


**Вывод**

Анализ данных показывает, что бездетные заемщики более дисциплинированные: 7% бездетных заемщиков не возвращают кредиты в срок, в то время как среди клиентов с детьми таких около 9-10%.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Сгруппируем данные по семейному положению и найдем сумму, количество и долю невозвратов для каждой группы.

In [51]:
family_status_grouped = data.groupby('family_status').agg({'debt': ['sum', 'count']})
family_status_grouped['frac'] = family_status_grouped['debt']['sum'] / family_status_grouped['debt']['count']
print(family_status_grouped)

                        debt             frac
                         sum  count          
family_status                                
Не женат / не замужем  274.0   2810  0.097509
в разводе               85.0   1195  0.071130
вдовец / вдова          63.0    959  0.065693
гражданский брак       388.0   4151  0.093471
женат / замужем        931.0  12339  0.075452


**Вывод**

Зависимость между семейным положением и возвратом кредита в срок проявляется в том, что:

- официально не состоящие в браке заемщики имеют б*о*льший процент невозвратов кредитов в срок в своей группе - около 9%;

- официально семейные и разведенные заемщики становятся должниками в 7% случаях;

- меньше всего просрочек по кредиту у одиноких людей (вдовых) - порядка 6%.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Сгруппируем данные по доходу и найдем сумму, количество и долю невозвратов для каждой группы.

In [52]:
income_grouped = data.groupby('income_group').agg({'debt': ['sum', 'count']})
income_grouped['frac'] = income_grouped['debt']['sum'] / income_grouped['debt']['count']
print(income_grouped)

                 debt             frac
                  sum  count          
income_group                          
высокий доход   120.0   1655  0.072508
низкий доход    438.0   5501  0.079622
средний доход  1183.0  14298  0.082739


**Вывод**

Заемщики со средним и низким доходом чаще всего имеют задолженности по кредиту. Что довольно предсказуемо.

- Как разные цели кредита влияют на его возврат в срок?

Сгруппируем данные по категориям целей и  найдем сумму, количество и долю невозвратов для каждой группы.

In [53]:
purpose_grouped = data.groupby('purpose_group').agg({'debt': ['sum', 'count']})
purpose_grouped['frac'] = purpose_grouped['debt']['sum'] / purpose_grouped['debt']['count']
print(purpose_grouped)

                debt             frac
                 sum  count          
purpose_group                        
автомобиль     403.0   4306  0.093590
недвижимость   782.0  10811  0.072334
образование    370.0   4013  0.092200
прочее         186.0   2324  0.080034


**Вывод**

Меньше всего задолженностей по кредитам возникает при операциях с недвижимостью.

<a id='conclusion'></a>
## Шаг 4. Общий вывод

Целью исследования было узнать, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок и каким образом.

Были проверены четыре гипотезы. Оценивалась **доля должников в каждой категории заемщиков**, поскольку исходные данные не равномерные и находить доли от общего количества клиентов в датасете неверно.

Анализ данных показал следующее:

1. Заемщики без детей более дисциплинированные: 7% бездетных заемщиков не возвращают кредиты в срок, в то время как среди клиентов с детьми таких около 9-10%.


2. Зависимость между семейным положением и возвратом кредита в срок проявляется в том, что:

- официально не состоящие в браке заемщики имеют б*о*льший процент невозвратов кредитов в срок в своей группе - около 9%;

- официально семейные и разведенные заемщики становятся должниками в 7% случаях;

- меньше всего просрочек по кредиту у одиноких людей (вдовых) - порядка 6%.

3. Величина дохода влияет на факт погашения кредита в срок: люди со средним и низким доходом чаще не возвращают кредиты в срок - таких около 8% в каждой группе.


4. Меньше всего задолженностей по кредитам возникает при операциях с недвижимостью (7% в своей группе), в то время как по кредитам на автомобили и образование не справляются 9% заемщиков.
